In [1]:
import  numpy   as  np
import  pytz
 
def initialize(context):
    context.ewa = sid(6683)
    context.ewc = sid(1787)
    
    context.delta = 0.0001 #delta
    context.Vw = context.delta/(1 - context.delta) * np.eye(2) #increase or decrease in price
    context. Ve  =   0.001
    context.beta = np.zeros(2) #beta
    context.P = np.zeros((2,2)) #predicted error of the system
    context.R  =   None #estimate of covariance
    
    context.pos = None #position
    context.day = None #current day
         
def handle_data(context, data):
    exchange_time = get_datetime().astimezone(pytz.timezone('US/Eastern'))
    # update Kalman filter and execute a trade during the final 5 minutes of trading for the day
    if  exchange_time.hour  ==   19   and  exchange_time.minute   >=   55: #anytime after 7:55pm Eastern
       # only execute this once per day
          if  context.day   is   not   None   and  context.day  == exchange_time.day:   #only once per day
              return
          context.day = exchange_time.day
          x = np.asarray([data[context.ewa].price,1.0]).reshape((1, 2))
          y = data [context.ewc].price
        
          if context.R is not None:
            context.R = context.P + context.Vw #update filter with latest price
          else:
            context.R = np.zeros((2,2))
          yhat = x.dot(context.beta) #initial prediction
        
          Q = x.dot(context.R).dot(x.T) + context.Ve #estimate of process error
          sqrt_Q = np.sqrt(Q) #establish signal bands
          e = y - yhat #estimator
          K = context.R.dot(x.T)/Q
          context.beta = context.beta + K.flatten()*e #calculate beta
          context.P = context.R - K*x.dot(context.R) #estimate error
        
          log.info(e)
        
          record(beta = context.beta[0], alpha = context.beta[1]) #difference between actual and expected 
        
          if e < 10:
            record(spread = float(e), Q_upper = float(sqrt_Q), Q_lower = float(-sqrt_Q))
            
        #do trades
        
        
          if context.pos is not None:
            order_target(context.ewa, 0)
            order_target(context.ewc, 0)
            #context.pos  =   None
                
          else:
            if e < -sqrt_Q:
                order (context.ewc, 1000)
                order (context.ewa,-1000*context.beta[0])
            #    context.pos = 'long'
                
            else:
                order (context.ewc, -1000 )
                order (context. ewa, 1000*context.beta[0])
               # context.pos = 'short'
                
        
        
        
        
       
        
        
        
            

